In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

DATA_DIR = '/content/drive/MyDrive/datathon_2022/data'

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# 1. Prepare dataframe

In [ ]:
# zip_info = pd.read_csv(os.path.join(DATA_DIR, 'zip_info.csv'), dtype={'zip3':str})

# df_acp = pd.read_pickle(os.path.join(DATA_DIR, 'accepted_merged.pickle'))
# df_acp = df_acp.drop(columns=['id']+zip_info.columns[3:].tolist())

# df_rej = pd.read_pickle(os.path.join(DATA_DIR, 'rejected.pickle'))

In [ ]:
emp2yr = {'10+ years':10, '3 years':3, '4 years':4, '6 years':6, '1 year':1, '7 years':7, '8 years':8, '5 years':5, '2 years':2, '9 years':9, '< 1 year':1, 0:0}
df_rej['emp_length'].replace(emp2yr, inplace=True)
df_acp['emp_length'].replace(emp2yr, inplace=True)
df_acp.rename(columns={'issue_d_year':'year'}, inplace=True)

In [ ]:
tdf = df_acp[df_rej.columns]
tdf['accepted'] = 1
tdf = pd.concat([tdf, df_rej], axis=0)
tdf['accepted'] = tdf['accepted'].fillna(0)

In [ ]:
del df_rej
gc.collect()

In [ ]:
tdf['dti'] = tdf['dti'].clip(upper=999)
tdf['dti'] = tdf['dti'].fillna(999)
tdf['dti_cat'] = pd.cut(tdf['dti'], bins=[0,20,40,100,1000], labels=["20","40", "100", "1000"])

In [ ]:
tdf.to_csv(os.path.join(DATA_DIR, 'simple_regress.csv'), index=False)

In [ ]:
# del tdf 
# gc.collect()

# 2. Logistic Regression

In [ ]:
df = pd.read_csv(os.path.join(DATA_DIR, 'simple_regress.csv'), dtype={'zip3': str, 'dti_cat': str})
zip_info = pd.read_csv(os.path.join(DATA_DIR, 'zip_info.csv'), dtype={'zip3':str})

In [ ]:
df = df[df['zip3'].isin(zip_info['zip3'].unique().tolist())]
df = df[df['loan_amnt'] >= 100]

In [ ]:
df['issue_q'] = pd.to_datetime(df['issue_d']).dt.to_period('Q')

df['post'] = (pd.to_datetime(df['issue_d']) > pd.to_datetime('6/1/2016')).astype(int)

black_zip = zip_info[(zip_info['is_black']>0.4) & (zip_info['year']==2016)]['zip3'].tolist()
df['black'] = (df['zip3'].isin(black_zip)).astype(int)
df['post_black'] = df['post'] * df['black']
df['log_loan_amnt'] = np.log(df['loan_amnt'])

In [ ]:
asian_zip = zip_info[(zip_info['is_black']>0.4) & (zip_info['year']==2016)]['zip3'].tolist()
df['black'] = (df['zip3'].isin(black_zip)).astype(int)

In [ ]:
df['log_loan_amnt'].plot.hist()

In [ ]:
import statsmodels.formula.api as sm

result = sm.logit(formula = "accepted ~ log_loan_amnt + C(dti_cat) + emp_length + C(post) + C(post_black) + C(year)", data = df).fit()

In [ ]:
result_2 = sm.logit(formula = "accepted ~ log_loan_amnt + C(dti_cat) + emp_length + C(post) + C(black) + C(post_black) + C(year)", data = df).fit()

In [ ]:
result.summary()

In [ ]:
# !pip install stargazer

In [ ]:
from stargazer.stargazer import Stargazer
out_stargazer = Stargazer([result])

In [ ]:
out_stargazer